<a href="https://colab.research.google.com/github/james-monahan/hopr-devrel/blob/QV-snapshot-game/analytics/QV-snapshot-game/HOPR_NFT_Tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports and Data

In [ ]:
import requests
import json
import pandas as pd
import time
import numpy as np

In [ ]:
#retrieve proposals
url = "https://hub.snapshot.org/graphql"

payload="{\"query\":\"query Proposals {\\r\\n  proposals(where: {space_in: [\\\"qvgame.hopr.eth\\\", \\\"HOPR DAO Voting Games\\\"], state: \\\"closed\\\"}, orderBy: \\\"created\\\", orderDirection: asc) {\\r\\n    id\\r\\n    scores\\r\\n    scores_by_strategy\\r\\n    scores_state\\r\\n    scores_total\\r\\n    scores_updated\\r\\n    votes\\r\\n    title\\r\\n    body\\r\\n    choices\\r\\n    start\\r\\n    end\\r\\n    snapshot\\r\\n    state\\r\\n    author\\r\\n    space {\\r\\n      id\\r\\n      name\\r\\n    }\\r\\n  }\\r\\n}\",\"variables\":{}}"
headers = {
  'Content-Type': 'application/json'
}

proposals = requests.request("POST", url, headers=headers, data=payload)

In [ ]:
#convert to dataframe and extract fields
proposals_data = json.loads(proposals.text)
proposals_df = pd.json_normalize(proposals_data["data"], "proposals" )
prop_id = proposals_df["id"].to_list()
prop_title = proposals_df["title"].to_list()
prop_choices = proposals_df["choices"].to_list()
prop_votes = proposals_df["votes"].to_list()
proposal_mapping = dict(zip(prop_id, prop_title))
choices_mapping = dict(zip(prop_id, prop_choices))
votes_mapping = dict(zip(prop_title, prop_votes))

In [ ]:
#retrive Votes data per proposal and create dataframe
votes_df = {}

for counter, proposal_ in enumerate(prop_id):
  url = "https://hub.snapshot.org/graphql"

  payload="{\"query\":\"query Votes {\\r\\n  votes(first: 1000, skip: 0, where: {proposal: \\\"zzzzzzzzzzzz\\\"}) {\\r\\n    id\\r\\n    proposal {\\r\\n      id\\r\\n    }\\r\\n    voter\\r\\n    created\\r\\n    choice\\r\\n    metadata\\r\\n    vp\\r\\n    vp_state\\r\\n    vp_by_strategy\\r\\n    space {\\r\\n      id\\r\\n    }\\r\\n  }\\r\\n}\",\"variables\":{}}".replace('zzzzzzzzzzzz', proposal_)
  headers = {
    'Content-Type': 'application/json'
  }

  votes = requests.request("POST", url, headers=headers, data=payload)
  votes_data = json.loads(votes.text)
  df = pd.json_normalize(votes_data["data"], "votes" )
  df["title"] = df["proposal.id"].map(proposal_mapping) 
  df["choices"] = df["proposal.id"].map(choices_mapping)
  votes_df[df["title"][0]] = df
  # votes_df.append(df)
  time.sleep(1)
  print(counter, end = '...')

0...1...2...3...4...5...6...7...8...9...10...

#NFT Level

In [ ]:
phase_one =  set(votes_df['Phase 1 Part 1: Find your team'].voter) &\
             set(votes_df['Phase 1 Part 2: Intro to QV'].voter) &\
             set(votes_df['Phase 1 Part 3: Split Choice QV'].voter)

In [ ]:
phase_two =  set(votes_df['Phase 2 Part 1: Simple Coordination'].voter) &\
             set(votes_df['Phase 2 Part 2: Complex Coordination'].voter) &\
             set(votes_df['Phase 2 Part 2: Disrupted Coordination 1'].voter) &\
             set(votes_df['Phase 2 Part 2: Disrupted Coordination 2'].voter)

In [ ]:
phase_three =  set(votes_df['Phase 3 Part 2: Adversarial Voting II'].voter) &\
               set(votes_df['Phase 3 Part 3: Adversarial Voting III'].voter)

# set(votes_df['Phase 3 Part 1: Adversarial Voting I'].voter) &\

In [ ]:
exclude = set(['0x29c8264AeB4B8cb0F14DEA198948A9cA54E0E828', '0xa27A79573ec108a23432Aa562e24541d8b691A6c',
               '0x90422fc0c841af68ae481AA5ABd7028BD126C1d3', '0x986E9e1bbd4D582C73aaD70DEDD8780EBA351D2A', 
               '0x8b4E5024CE949Ff05084f211658719929E3Bea71', '0xf38778ad9f2a401395A54A6c47e3e91F39bc028b', 
               '0x931cC23c0660789AB17B1c722341A6C811985Ed3', '0x12138Db1E794B20D21266c3E6CcdabCFa2680750', 
               '0xA45a5c1333B7776ef04844887ad22428C145Ad1C', '0x801C864c4890DdE8A4950A8075717DB48E78b70f', 
               '0x51E4dEcE3f065c380A96702082EBD468a6Db6E0A', '0x321302B0Af369F5c93c6b0AAE6eF21DaDe211044', 
               '0x269A0266CcDee7c23833B5E11443D5C3758a290a', '0xb7c1b4890ee41a00Fe5a0b97714f049276935bA6'])

In [ ]:
def assign_result(df):
  if df["phase_one"] + df["phase_two"] + df["phase_three"] == 3:
    return 'diamond'
  elif df["phase_one"] + df["phase_two"] + df["phase_three"] == 2:
    return 'gold'
  elif df["phase_one"] + df["phase_two"] + df["phase_three"] == 1:
    return 'silver'
  else:
    return None

In [ ]:
all_address = phase_one | phase_two | phase_three
all_address = all_address - exclude

rewards_df = pd.DataFrame(list(all_address), columns=['eoa'])

rewards_df["phase_one"] = rewards_df["eoa"].apply(lambda x: x in phase_one)
rewards_df["phase_two"] = rewards_df["eoa"].apply(lambda x: x in phase_two)
rewards_df["phase_three"] = rewards_df["eoa"].apply(lambda x: x in phase_three)

rewards_df["grade"] = rewards_df.apply(lambda x: assign_result(x), axis=1)

rewards_df = rewards_df[['eoa', 'grade', 'phase_one', 'phase_two', 'phase_three']]

rewards_df["grade"].value_counts()

diamond    214
gold        65
silver      43
Name: grade, dtype: int64

In [ ]:
rewards_df

,eoa,grade,phase_one,phase_two,phase_three
0,0xEd534F3F4CDF9df19C3082189964E5e8D162D862,diamond,True,True,True
1,0xd24c941f5Edbe451395544658c0dF6dF056c6aD0,diamond,True,True,True
2,0x35e939c53B208CCec114c6b92f6d754b023dff4D,diamond,True,True,True
3,0xe1Da425d1880D3243f9Dfd3c9C2C3f09E73ACd6d,diamond,True,True,True
4,0x617C716cd4eeF3D20d5a0CACAC9518647Bb19173,diamond,True,True,True
...,...,...,...,...,...
317,0x9062A96B9E947b2b57283e705c97B871f5eB6561,diamond,True,True,True
318,0x852926150De3854D61319021BD88F239C1Cc4C38,diamond,True,True,True
319,0xd164984268bC6e81Ce9a5109C955b96E4edBEE89,silver,True,False,False
320,0x93803d334edFF050EB333F0337Ac0E728932bAA3,diamond,True,True,True


#Token Distributions

https://snapshot.org/#/qvgame.hopr.eth

##Phase 1 Part 1: Find your team 
Prize: 1000 HOPR Tokens (split evenly amongst correct answers)  
(Exclude) Anyone who votes for the wrong team option

In [ ]:
P1P1_TeamA = votes_df['Phase 1 Part 1: Find your team'][votes_df['Phase 1 Part 1: Find your team'].vp==100]
P1P1_TeamB = votes_df['Phase 1 Part 1: Find your team'][votes_df['Phase 1 Part 1: Find your team'].vp==10_000]
P1P1_TeamC = votes_df['Phase 1 Part 1: Find your team'][votes_df['Phase 1 Part 1: Find your team'].vp==1_000_000]

P1P1 = pd.DataFrame(
       list(P1P1_TeamA[P1P1_TeamA['choice'] == 1].voter) +\
       list(P1P1_TeamB[P1P1_TeamB['choice'] == 2].voter) +\
       list(P1P1_TeamC[P1P1_TeamC['choice'] == 3].voter),
       columns= ['eoa'])
P1P1['Phase 1 Part 1: Find your team'] = 1000/P1P1.shape[0]

rewards_df = rewards_df.merge(P1P1, how='outer', on='eoa')

##Phase 1 Part 2: Intro to QV
Prize: 1000 HOPR Tokens (split evenly amongst correct answers)  
(Exclude) Anyone who votes for the wrong team option or splits vote across multiple teams

In [ ]:
P1P2_TeamA = votes_df['Phase 1 Part 2: Intro to QV'][votes_df['Phase 1 Part 2: Intro to QV'].vp==100]
P1P2_TeamB = votes_df['Phase 1 Part 2: Intro to QV'][votes_df['Phase 1 Part 2: Intro to QV'].vp==10_000]
P1P2_TeamC = votes_df['Phase 1 Part 2: Intro to QV'][votes_df['Phase 1 Part 2: Intro to QV'].vp==1_000_000]

P1P2 = pd.DataFrame(
            list(P1P2_TeamA[P1P2_TeamA[['choice.2', 'choice.3']].sum(axis=1)==0].voter) +\
            list(P1P2_TeamB[P1P2_TeamB[['choice.1', 'choice.3']].sum(axis=1)==0].voter) +\
            list(P1P2_TeamC[P1P2_TeamC[['choice.1', 'choice.2']].sum(axis=1)==0].voter),
            columns= ['eoa'])

P1P2['Phase 1 Part 2: Intro to QV'] = 1000/P1P2.shape[0]

rewards_df = rewards_df.merge(P1P2, how='outer', on='eoa')

##Phase 1 Part 3: Split Choice QV  
Prize: 
- The vote count of the least popular option will be the final score and determine the prize payout.   
- Up to 5000 HOPR tokens (split evenly)
- No wrong choices  

**vote count = (% of choice * thier number of votes)**



In [ ]:
pd.options.mode.chained_assignment = None
P1P3 = votes_df['Phase 1 Part 3: Split Choice QV'][['voter', 'vp','choice.1', 'choice.2', 'choice.3', 'choice.4', 'choice.5']]
P1P3["total"] = P1P3[['choice.1', 'choice.2', 'choice.3', 'choice.4', 'choice.5']].sum(axis=1)
P1P3['choice.1'] = (P1P3['choice.1']/P1P3['total']) * P1P3['vp']
P1P3['choice.2'] = (P1P3['choice.2']/P1P3['total']) * P1P3['vp']
P1P3['choice.3'] = (P1P3['choice.3']/P1P3['total']) * P1P3['vp']
P1P3['choice.4'] = (P1P3['choice.4']/P1P3['total']) * P1P3['vp']
P1P3['choice.5'] = (P1P3['choice.5']/P1P3['total']) * P1P3['vp']
print("Vote count of the least popular option:",min(P1P3['choice.1'].sum(), P1P3['choice.2'].sum(), P1P3['choice.3'].sum(), P1P3['choice.4'].sum(), P1P3['choice.5'].sum()))

Vote count of the least popular option: 142944.2929155997


###Least popular count exceeds 5,000, prize is 5,000

In [ ]:
P1P3 = P1P3[['voter']]
P1P3.columns = ['eoa']
P1P3['Phase 1 Part 3: Split Choice QV'] = 5000/P1P3.shape[0]
rewards_df = rewards_df.merge(P1P3, how='outer', on='eoa')

##Phase 2, Part 1 Simple Coordination:
Prize: 
- 2000 HOPR Tokens (split evenly)   
- No wrong choices


In [ ]:
#Option 1 > Option 2 > Option 3 > Option 4 > Option 5
#verify option 5 minimum vote count
votes_df['Phase 2 Part 1: Simple Coordination'].vp.sum() * .0972

67252.68

In [ ]:
P2P1 = pd.DataFrame(
            list(votes_df['Phase 2 Part 1: Simple Coordination'].voter),
            columns= ['eoa'])

P2P1['Phase 2 Part 1: Simple Coordination'] = 2000/P2P1.shape[0]
rewards_df = rewards_df.merge(P2P1, how='outer', on='eoa')

##Phase 2 Part 2: Complex Coordination  
Prize: 
- 2500 HOPR Tokens (split evenly)   
- No wrong choices

In [ ]:
#verify less than 1% margin of error
(30-29.92) + (25-24.84) + (20-19.79) + (15.44-15) + (10.02-10)

0.9099999999999984

In [ ]:
P2P2 = pd.DataFrame(
            list(votes_df['Phase 2 Part 2: Complex Coordination'].voter),
            columns= ['eoa'])

P2P2['Phase 2 Part 2: Complex Coordination'] = 2500/P2P2.shape[0]
rewards_df = rewards_df.merge(P2P2, how='outer', on='eoa')

##Phase 2 Part 2: Disrupted Coordination 1
Prize: 
- 2000 HOPR Tokens (split evenly)   
- (Exclude) Team A member voting for Option 3 or 4
- (Exclude)Team B member voting for Option 1 or 2

In [ ]:
#verify 5% error margin
four = 48.29
three = 39.05 
two = 8.43 
one = 4.23
#5% margin error
# Team A: Make Option 1 receive half as many votes as Option 2.
# Team B: Make Option 3 receive 10% fewer votes than Option 4

2-(two/one), (four*.9)-three

(0.007092198581560627, 4.411000000000001)

In [ ]:
P2P2P1_TeamA = votes_df['Phase 2 Part 2: Disrupted Coordination 1'][votes_df['Phase 2 Part 2: Disrupted Coordination 1'].vp == 100] 
P2P2P1_TeamB = votes_df['Phase 2 Part 2: Disrupted Coordination 1'][votes_df['Phase 2 Part 2: Disrupted Coordination 1'].vp == 10000]


P2P2P1 = pd.DataFrame(
            (list(P2P2P1_TeamA[P2P2P1_TeamA[["choice.3", "choice.4"]].sum(axis=1)==0].voter) +\
             list(P2P2P1_TeamB[P2P2P1_TeamB[["choice.1", "choice.2"]].sum(axis=1)==0].voter)),
             columns= ['eoa'])

P2P2P1['Phase 2 Part 2: Disrupted Coordination 1'] = 2000/P2P2P1.shape[0]
rewards_df = rewards_df.merge(P2P2P1, how='outer', on='eoa')

##Phase 2 Part 2: Disrupted Coordination 2
Prize: 
- 2000 HOPR Tokens (for whichever team gets closest to achieving their goal)   
- (Exclude) Team A (Didn't win goal)
- (Exclude) Team B member voting for Option 1



In [ ]:
three = 61.68 
two = 32.27 
one = 6.05
#Goals
# A Make Option 1 and Option 2 combined receive 40% of the vote.
# B Make Option 3 receive twice as many votes as Option 2
(40 - (one+two)), (2 - (three/two))

(1.6799999999999997, 0.088627207933065)

In [ ]:
P2P2P2_TeamA = votes_df['Phase 2 Part 2: Disrupted Coordination 2'][votes_df['Phase 2 Part 2: Disrupted Coordination 2'].vp == 100] 
P2P2P2_TeamB = votes_df['Phase 2 Part 2: Disrupted Coordination 2'][votes_df['Phase 2 Part 2: Disrupted Coordination 2'].vp == 10000]

P2P2P2 =  pd.DataFrame(
              list(P2P2P2_TeamB[P2P2P2_TeamB["choice.1"].isna()].voter),
               columns= ['eoa']) 

P2P2P2['Phase 2 Part 2: Disrupted Coordination 2'] = 2000/P2P2P2.shape[0]
rewards_df = rewards_df.merge(P2P2P2, how='outer', on='eoa')

##Phase 3 Part 1: Adversarial Voting I
Prize: 
- 2000 HOPR Tokens (split evenly)   
- (Exclude) Anyone voting for Option 1 or 2

In [ ]:
#100% will be awarded if Option 1 and 2 come in the bottom two places: TRUE

In [ ]:
P3P1 = pd.DataFrame(
              list(votes_df['Phase 3 Part 1: Adversarial Voting I']\
                  [votes_df['Phase 3 Part 1: Adversarial Voting I'][['choice.1', 'choice.2']].sum(axis=1) == 0].voter),
                  columns= ['eoa']) 

P3P1 = P3P1[P3P1['eoa'] != '0x29c8264AeB4B8cb0F14DEA198948A9cA54E0E828']

P3P1['Phase 3 Part 1: Adversarial Voting I'] = 2000/P3P1.shape[0]
rewards_df = rewards_df.merge(P3P1, how='outer', on='eoa')

##Phase 3 Part 2: Adversarial Voting II
Prize: 
- 3000 HOPR Tokens (split evenly)   
- (Exclude) Anyone voting for Option 1 or 2

In [ ]:
# 100% will be awarded if Option 1 and 2 come in the bottom two places: TRUE

In [ ]:
P3P2 = pd.DataFrame(
              list(votes_df['Phase 3 Part 2: Adversarial Voting II']\
              [votes_df['Phase 3 Part 2: Adversarial Voting II'][['choice.1', 'choice.2']].sum(axis=1) == 0].voter),
               columns= ['eoa']) 

P3P2['Phase 3 Part 2: Adversarial Voting II'] = 3000/P3P2.shape[0]
rewards_df = rewards_df.merge(P3P2, how='outer', on='eoa')

##Phase 3 Part 3: Adversarial Voting III
Prize: 
- 7000 HOPR Tokens 
- 4000 will be awarded if Option 3 or 4 wins. 
- An additional 3000 will be awarded to the team whose proposal wins (Team B with Option 4)
- (Exclude) Anyone voting for Option 1 or 2

In [ ]:
#Option 4 wins, Team B

In [ ]:
P3P3 = pd.DataFrame(
              list(
                  votes_df['Phase 3 Part 3: Adversarial Voting III']\
                 [votes_df['Phase 3 Part 3: Adversarial Voting III'][['choice.1', 'choice.2']].sum(axis=1) == 0].voter
                  ),
              columns= ['eoa']) 

P3P3_bonus = pd.DataFrame(
              list(
                  votes_df['Phase 3 Part 3: Adversarial Voting III']\
                  [(votes_df['Phase 3 Part 3: Adversarial Voting III'][['choice.1', 'choice.2']].sum(axis=1) == 0) &\
                  (votes_df['Phase 3 Part 3: Adversarial Voting III']['vp']==10000)].voter
                  ),
              columns= ['eoa']) 

P3P3['Phase 3 Part 3: Adversarial Voting III'] = 4000/P3P3.shape[0]
P3P3_bonus['Phase 3 Part 3: Adversarial Voting III BONUS'] = 3000/P3P3_bonus.shape[0]

rewards_df = rewards_df.merge(P3P3, how='outer', on='eoa')
rewards_df = rewards_df.merge(P3P3_bonus, how='outer', on='eoa')

##Phase 3 Part 4: Disruptive Voting
Prize: 
- 7000 HOPR Tokens 
- 4000 will be awarded if Either team wins. 
- An additional 3000 will be awarded to the team whose proposal wins (Team B with Options 4,5,6,8)
- No wrong choices

In [ ]:
#Options 4,5,6,8 win

# Team B wants TWO of Options 4, 5 and 6 to place in the top, and Option 8 to NOT place in the top 3.

In [ ]:
P3P4 = pd.DataFrame(
              list(votes_df['Phase 3 Part 4: Disruptive Voting'].voter),
              columns= ['eoa']) 

P3P4_bonus = pd.DataFrame(
              list(
                  votes_df['Phase 3 Part 4: Disruptive Voting']\
                 [votes_df['Phase 3 Part 4: Disruptive Voting']['vp']==10000].voter
                  ),
              columns= ['eoa']) 

P3P4['Phase 3 Part 4: Disruptive Voting'] = 4000/P3P4.shape[0]
P3P4_bonus['Phase 3 Part 4: Disruptive Voting BONUS'] = 3000/P3P4_bonus.shape[0]

rewards_df = rewards_df.merge(P3P4, how='outer', on='eoa')
rewards_df = rewards_df.merge(P3P4_bonus, how='outer', on='eoa')

#Rewards

###NFT

In [ ]:
rewards_df[rewards_df['grade'].isna()]

In [ ]:
HOPR_NFT = rewards_df[['eoa', 'grade']]
HOPR_NFT["eoa"] = HOPR_NFT["eoa"].apply(lambda x: ">"+x+"<")
print(HOPR_NFT[HOPR_NFT.grade.isna()].shape)
print(HOPR_NFT.shape)
HOPR_NFT.dropna(subset=['grade'], inplace = True)
print(HOPR_NFT.shape)

(5, 2)
(327, 2)
(322, 2)


In [ ]:
HOPR_NFT['grade'].value_counts(dropna=False) 

diamond    214
gold        65
silver      43
Name: grade, dtype: int64

In [ ]:
HOPR_NFT.to_csv('./HOPR_NFT.csv', index=False)

In [ ]:
! head HOPR_NFT.csv

eoa,grade
>0xa2AB20E4aA2Cfc44231161C005cBF6fD68BF56a9<,diamond
>0x960cC811e5eF36760f5c679AF90e821B10385e04<,diamond
>0xdAf08467BE187fDAD0DB1df64b8cbD20F60AB370<,diamond
>0xb5cBB2f3B5d3A72a88aF6c3739579A18D5235755<,diamond
>0x205c211A68E0b8C1a97D61b0B09099231b3C160D<,silver
>0x6BcFB1F96A3d270F00dEAb39Fd50e78096576113<,silver
>0x3f4d0a6654c8365B42DcAa8AA92d9034e2A6141f<,diamond
>0xbE93d14C5dEFb8F41aF8FB092F58e3C71C712b85<,gold
>0xBE8C93a8C18AF63aAB449994AFAc13E71240ccC4<,diamond


###Tokens

In [ ]:
rewards_df.columns

In [ ]:
rewards_cols = ['Phase 1 Part 1: Find your team', 
                'Phase 1 Part 2: Intro to QV',
                'Phase 1 Part 3: Split Choice QV',
                'Phase 2 Part 1: Simple Coordination',
                'Phase 2 Part 2: Complex Coordination',
                'Phase 2 Part 2: Disrupted Coordination 1',
                'Phase 2 Part 2: Disrupted Coordination 2',
                'Phase 3 Part 1: Adversarial Voting I',
                'Phase 3 Part 2: Adversarial Voting II',
                'Phase 3 Part 3: Adversarial Voting III',
                'Phase 3 Part 3: Adversarial Voting III BONUS',
                'Phase 3 Part 4: Disruptive Voting',
                'Phase 3 Part 4: Disruptive Voting BONUS']

In [ ]:
rewards_df

In [ ]:
HOPR_tokens = rewards_df
HOPR_tokens['rewards_sum'] = HOPR_tokens[rewards_cols].sum(axis=1)
HOPR_tokens = HOPR_tokens[['eoa', 'rewards_sum']]
HOPR_tokens.to_csv('./HOPR_tokens.csv', index=False)

In [ ]:
HOPR_tokens['rewards_sum'].sum()

34500.0

In [ ]:
! head HOPR_tokens.csv

eoa,rewards_sum
0xa2AB20E4aA2Cfc44231161C005cBF6fD68BF56a9,63.13109143778331
0x960cC811e5eF36760f5c679AF90e821B10385e04,106.61653000971766
0xf7c6382Da0A3ee08aD4c201b01E33e122600Ab6f,82.82782088625194
0xdAf08467BE187fDAD0DB1df64b8cbD20F60AB370,80.84061190767022
0xb5cBB2f3B5d3A72a88aF6c3739579A18D5235755,47.06683440967086
0x205c211A68E0b8C1a97D61b0B09099231b3C160D,116.79451276525594
0x6BcFB1F96A3d270F00dEAb39Fd50e78096576113,35.78580108379298
0x3f4d0a6654c8365B42DcAa8AA92d9034e2A6141f,272.86548573235757
0xbE93d14C5dEFb8F41aF8FB092F58e3C71C712b85,48.46022760343085


#Supplemental

###GraphQL queries

In [ ]:
# query Votes {
#   votes(first: 1000, skip: 0, where: {proposal: "0x5ba405215e79898f8ec1c62112f03094933e64b81446b3066531ee31bc0b31d0"}) {
#     id
#     proposal {
#       id
#     }
#     voter
#     created
#     choice
#     metadata
#     vp
#     vp_state
#     vp_by_strategy
#     space {
#       id
#     }
#   }
# }

In [ ]:
# query Proposals {
#   proposals(where: {space_in: ["qvgame.hopr.eth", "HOPR DAO Voting Games"], state: "closed"}, orderBy: "created", orderDirection: asc) {
#     id
#     scores
#     scores_by_strategy
#     scores_state
#     scores_total
#     scores_updated
#     votes
#     title
#     body
#     choices
#     start
#     end
#     snapshot
#     state
#     author
#     space {
#       id
#       name
#     }
#   }
# }

###Data exports - raw

In [ ]:
#create csv file of all data
proposals_df.to_csv('./proposals_df.csv', index=False)

for title in votes_df.keys():
  votes_df[title].to_csv(f'./{title}.csv', index=False)